In [2]:
#!pip install biopython
from Bio import SeqIO 
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord

In [3]:
import numpy as np

In [4]:
reads = list(SeqIO.parse('single_Pfal_dat.fq','fastq'))

In [5]:
reads

[SeqRecord(seq=Seq('TTTCCTTTTTAAGCGTTTTATTTTTTAATAAAAAAAATATAGTATTATATAGTA...TAA'), id='NC_004325.2-100000', name='NC_004325.2-100000', description='NC_004325.2-100000', dbxrefs=[]),
 SeqRecord(seq=Seq('TATATCTTTAAAATGATGTTGCAAATTTATTGAACATGTTAATAAATCATCCTG...TCA'), id='NC_004325.2-99999', name='NC_004325.2-99999', description='NC_004325.2-99999', dbxrefs=[]),
 SeqRecord(seq=Seq('TCATGATTTACATATATTTGTAAAACATATATAATCTGTCCAGACATATTATAT...CTC'), id='NC_004325.2-99998', name='NC_004325.2-99998', description='NC_004325.2-99998', dbxrefs=[]),
 SeqRecord(seq=Seq('AATTAATAATTATTATTATTACATATATATTTGTTATGTTTTGTTATATAATAT...TGT'), id='NC_004325.2-99997', name='NC_004325.2-99997', description='NC_004325.2-99997', dbxrefs=[]),
 SeqRecord(seq=Seq('ATGCTTTTTAAAAATGAAGACAGTGGCAATGGTGTTTACGGTTTTACTTATAAT...ATG'), id='NC_004325.2-99996', name='NC_004325.2-99996', description='NC_004325.2-99996', dbxrefs=[]),
 SeqRecord(seq=Seq('AAAATATTAATATAGAAATGTTAAAAATACGAATCAATTATATTTGTGATAAAG...TAT'), id='NC_004325

In [6]:
Genome = list(SeqIO.parse('data','fasta'))

In [7]:
Genome

[SeqRecord(seq=Seq('TGAACCCTaaaacctaaaccctaaaccctaaaccctgaaccctaaaccctgaac...agg'), id='NC_004325.2', name='NC_004325.2', description='NC_004325.2 Plasmodium falciparum 3D7 genome assembly, chromosome: 1', dbxrefs=[]),
 SeqRecord(seq=Seq('aaccctaaaccctaaaccctaaaccctaaaccctaaaccctaaacctaaaccct...TCA'), id='NC_037280.1', name='NC_037280.1', description='NC_037280.1 Plasmodium falciparum 3D7 genome assembly, chromosome: 2', dbxrefs=[]),
 SeqRecord(seq=Seq('TAAACCCTAAATCTCTAAACCCTAAAGCTATACCTAAACCCTGAAGGTTATACC...TCA'), id='NC_000521.4', name='NC_000521.4', description='NC_000521.4 Plasmodium falciparum 3D7 genome assembly, chromosome: 3', dbxrefs=[]),
 SeqRecord(seq=Seq('aaccctaaaccctgaaccctaaaccctaaaccctgaaccctgaaccctaaaccc...tta'), id='NC_004318.2', name='NC_004318.2', description='NC_004318.2 Plasmodium falciparum 3D7 genome assembly, chromosome: 4', dbxrefs=[]),
 SeqRecord(seq=Seq('ctaaaccctgaaccctaaaccctgaaccctaaaccctaaaccctgaaccctaaa...ggt'), id='NC_004326.2', name='NC_004326.2', de

Pour élaborer votre méthode de mapping, il est nécessaire de pouvoir lire et analyser 
des fichiers de données de séquençage. La première étape du projet consiste donc à prendre 
en main la bibliothèque Biopython pour manipuler les fichiers au format FASTQ.

Dans un second temps, il vous faudra définir une structure de données permettant de 
chercher des mots de longueur fixe (k > 0) à partir d’un texte t de longueur n et capable de 
répondre efficacement aux questions suivantes : 
-  Étant donné un mot w de longueur k, est-il présent (lui ou son complémentaire inversé) dans le texte indexé t ? 
    - le cas échéant, combien de fois apparaît-il (quel est son support), à quelles positions et sur quel brin ? 
 
-  Quel est le mot de longueur k présent à la position i (0<i<=n) dans le texte t ?
 
-  Bonus : proposez une méthode de recherche de sous-séquences approchée autorisant l’alignement de sous-séquences avec un nombre maximal d’erreurs d. 
 
   
Dans le choix de la structure, vous veillerez bien à prendre en compte le fait que : 
1.  le texte t est formé sur un petit alphabet (A, C, G et T) 
2.  le texte t peut-être très grand (n > 109) 
3.  une structure de données peut être constituée de plusieurs structures distinctes.

Ajouter "$" à la fin des séquences.

In [7]:
end = Seq('$')
G = [s.upper() for s in Genome]
for i in range(len(Genome)):
    G[i].seq = Genome[i].seq+end
    

In [8]:
len(G[0].seq)

640852

In [9]:
len(Genome[0].seq)

640851

Test sur la première seqence du génome.

In [10]:
s = G[0].seq
print(len(s))
str(s[0:100])

640852


'TGAACCCTaaaacctaaaccctaaaccctaaaccctgaaccctaaaccctgaaccctaaaccctaaaccctgaaccctaaaccctaaaccctgaacccta'

Dans un second temps, il vous faudra définir une structure de données permettant de chercher des mots de longueur fixe (`k > 0`) à partir d’un texte `t` de longueur `n` et capable de répondre efficacement aux questions suivantes :

-  Étant donné un mot `w` de longueur `k`, est-il présent (lui ou son complémentaire inversé) dans le texte indexé `t` ? 
    
    - le cas échéant, combien de fois apparaît-il (quel est son support), à quelles positions et sur quel brin ? 
 
-  Quel est le mot de longueur `k` présent à la position `i` dans le texte `t` ?

In [13]:
def complementInverse(w):
    wCI = []
    for i in range(len(w)):
        li = w[-i-1]
        if li not in "ATCG":
            raise Exception("Letter not in alphabet")
        lCI = 'A'*int(li=='T') + 'T'*int(li=='A') + 'C'*int(li=='G') + 'G'*int(li=='C')
        wCI.append(lCI)
    return ''.join(wCI)

In [14]:
testWord = "ATCG"
ansWord = "CGAT"
ans = complementInverse(testWord)
if complementInverse(testWord) != ansWord:
    raise Exception("Wrong result")

In [15]:
def circularShift(T,k):
    if k<0:
        raise Exception("k must be positive")
    return T[k:]+T[0:k]

def BWT(S):
    if S[len(S)-1]!='$':
        S += '$'
        
    circShifts = []
    for k in range(len(S)):
        circShifts.append(circularShift(S,k))
    circShifts.sort()
    
    bwt = []
    for shift in circShifts:
        bwt.append(shift[len(S)-1])
        
    return bwt

In [16]:
testText = "ATGCCTGATCG"
testTextBWT = BWT(testText)

In [17]:
testTextBWT

['G', 'G', '$', 'G', 'T', 'C', 'C', 'T', 'T', 'A', 'C', 'A']

In [18]:
def suffix_list(T): # O(n log n)
    """
    Compute the suffix list
    
    Args:
        T (str): string
    
    Return:
        list of strings: suffix list
    """
    suffix_list = [T[i:] for i in range(len(T))] # O(n)
    sorted(suffix_list,reverse=True) # O(n log n)
    return suffix_list

def suffix_table(T): # O(n log n)
    """
    Compute the suffix table
    
    Args:
        T (str): string
    
    Return:
        list of tuples (suffix,location): suffix table
    """
    suffix_list = [T[i:] for i in range(len(T))] 
    suffix_table = sorted((e,i) for i,e in enumerate(suffix_list))
    return suffix_table

def BWT_suffix_table(T,end_of_string="$"): # O(n log n)
    """
    Compute the BWT from the suffix table
    
    Args:
        T (str): string
        end_of_string (char): end of string character to append
    
    Return:
        bwt (str): BWT
    """
    if T[-1]!=end_of_string :
        T += end_of_string
    ST = suffix_table(T) # O(n log n)
    bwt = ""
    index = []
    for s,i in ST: # O(n)
        bwt += T[i-1]
        index.append(i-1)
    return bwt, index

Let `n` the size of `T`.

Complexity of `suffix_list(T)` :

In [17]:
suffix_list(testText)

['ATGCCTGATCG',
 'TGCCTGATCG',
 'GCCTGATCG',
 'CCTGATCG',
 'CTGATCG',
 'TGATCG',
 'GATCG',
 'ATCG',
 'TCG',
 'CG',
 'G']

In [18]:
suffix_table(testText)

[('ATCG', 7),
 ('ATGCCTGATCG', 0),
 ('CCTGATCG', 3),
 ('CG', 9),
 ('CTGATCG', 4),
 ('G', 10),
 ('GATCG', 6),
 ('GCCTGATCG', 2),
 ('TCG', 8),
 ('TGATCG', 5),
 ('TGCCTGATCG', 1)]

In [19]:
BWT_suffix_table(testText)

('GG$GTCCTTACA', [10, 6, -1, 2, 8, 3, 9, 5, 1, 7, 4, 0])

In [19]:
from collections import Counter 

def occurrence_indexer(S):
    """
    Number of past occurrences of each char in S
    
    Args:
        S (str): string
        
    Return:
        table of int : number of past occurrences of the char
    """
    K = []
    last_index = {}
    for s in S:
        if s not in last_index: 
            last_index[s] = 0
        K.append(last_index[s])
        last_index[s] += 1
    return(K)

def last2first(counts,k,X):
    """
    k + nbr of occurrences of letters < X 
    
    Args:
        counts (Counter): nbr of each char in the text
        k : int
        X : char
        
    Return:
        int : k + nbr of occurrences of letters < X 
    """
    a = [counts[char] for char in counts if char < X]
    return k + sum(a)
    
def efficient_inverse_BWT(bwt,end_of_string="$"):
    """
    Inverse the BWT
    
    Args:
        bwt (str): bwt of a string T
        last_character (char): which is the end of string character?
    
    Return:
        T (str): BWT^{-1} of bwt
    """    
    K = occurrence_indexer(bwt)
    counts = Counter(bwt)
    X = bwt[0]
    k = K[0]
    S = end_of_string
    while X != end_of_string:
        S = X+S
        j = last2first(counts,k,X)
        X = bwt[j]
        k = K[j]
    return(S)

In [21]:
occurrence_indexer(testText)

[0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2]

In [22]:
testText

'ATGCCTGATCG'

In [23]:
last2first(Counter(testText),0,"A")

0

In [24]:
#efficient_inverse_BWT(BWT_suffix_table(testText)) == testText + "$"

In [20]:
  def get_first_occurrence(L,X):
    for i,l in enumerate(L):
        if l == X:
            return(i)
        
def pattern_matching_BWT(T,pattern):
    """
    Search a patter in a String using the BWT
    
    Args:
        S (str): string
        pattern (str): pattern
    
    Return:
        bool: true if the pattern is in the string    
    """
    L,_ = BWT_suffix_table(T)
    K = occurrence_indexer(L)
    counts = Counter(L)
    e = 0
    f = len(L)
    i = len(pattern) - 1
    while e < f and i >= 0:
        X = pattern[i]
        first_occurence_in_L_ef = get_first_occurrence(L[e:f],X)
        if first_occurence_in_L_ef is None:
            return False
        else:
            r = first_occurence_in_L_ef+e
        #print(first_occurence_in_L_ef,L[e:f],r,X,L[r])
        
        last_occurence_in_L_ef = get_first_occurrence(L[e:f][::-1],X)
        if last_occurence_in_L_ef is None:
            return False
        else:
            s = f-last_occurence_in_L_ef-1
        #print(last_occurence_in_L_ef,L[e:f],s,X,L[s])
        
        e = last2first(counts,K[r],X)
        f = last2first(counts,K[s],X)+1
        i -= 1
        #print(r,s,e,f)
    return(i<0)



In [21]:
pattern_matching_BWT(testText,testWord)

True

In [22]:
def rankBWT(bwt):
    """
    Number of past occurrences of each char in S
    
    Args:
        S (str): string
        
    Return:
        table of int : number of past occurrences of the char
    """
    K = []
    tot = {}
    for s in bwt:
        if s not in tot: 
            tot[s] = 0
        K.append(tot[s])
        tot[s] += 1
    return(K,tot)
    

In [28]:
testBWT,_ = BWT_suffix_table(testText)

In [29]:
rank,tots = rankBWT(testBWT)

In [23]:
def invBWT(bwt,A):
    A = sorted(A)
    #count = {a:0 for a in A}
    count = {}
    K = []
    for i in range(len(bwt)):
        if bwt[i] in count:
            count[bwt[i]] += 1
        else:
            count[bwt[i]] = 1
        K.append(count[bwt[i]])
    X = bwt[0]
    k = K[1]
    S = '$'
    while X != '$':
        S = X+S
        j = k
        i = 0
        while A[i] != X:
            j += count[A[i]]
            i += 1
        X = bwt[j]
        k = K[j]
    return S

In [24]:
def firstColMap(tots):
    """ Return a map from characters to the range of cells in the first
    column containing the character. 
    char : (first index, last index+1)"""
    first = {}
    totc = 0
    for c, count in sorted(tots.items()):
        first[c] = (totc, totc + count)
        totc += count
    return first

In [25]:
import sys

In [26]:
def searching(bwt, SAT, W):
    
    # Not case sensitive, make everything uppercase
    bwt = bwt.upper()
    W = W.upper()
    
    # Perdorm a BWT on text T and get the indexes corresponding to the Suffix Array of T
    #bwt, SAT = BWT_suffix_table(T)
    # LF mapping
    # F = sorted(bwt) # no need to store it
    rank, totCount = rankBWT(bwt)
    Fmap = firstColMap(totCount)
    del totCount
    
    #print('\nbwt :', sys.getsizeof(bwt),
    #'\nSAT :', sys.getsizeof(SAT),
    #'\nrank :', sys.getsizeof(rank),
    #'\nFmap :', sys.getsizeof(Fmap),)

    letter = W[-1] # last letter of the word W
    if letter not in Fmap:
        return -1, "Word not in text"
    fi, li = Fmap[letter] # find occurences of letter in F
    
    for i in range(2,len(W)+1):
        letter = W[-i] # search W by going backward through it
        if letter not in Fmap:
            return -1, "Word not in text"

        j = fi
        foundRankf = False
        while j < li:
            if bwt[j] == letter:
                if not foundRankf:
                    rankf = rank[j] # get the rank of the first occurence of letter (followed by W suffix)
                    foundRankf = True
                rankl = rank[j] # get the rank of the last occurence of letter (followed by W suffix)
            j += 1
        
        if not foundRankf:
            return -1, "Word not in text"
        
        fi, li = Fmap[letter] # find occurences of letter in F
        fi += rankf # find first occurence of letter in F (followed by W suffix)
        li = fi + rankl-rankf +1 # find last occurence of letter in F (followed by W suffix)
        
    return [x+1 for x in SAT[fi:li]]
    

In [34]:
BWTpwet, SATpwet = BWT_suffix_table('pwetPWetpwotpwitpwet')
searching(BWTpwet, SATpwet, 'PWET')

(-1, 'Word not in text')

In [35]:
# sWord = 'c'
# s10 = searching(str(s)[0:10], sWord)
# s100 = searching(str(s)[0:100], sWord)
# s1000 = searching(str(s)[0:1000], sWord)

## Browse reads

In [36]:
reads

[SeqRecord(seq=Seq('TTTCCTTTTTAAGCGTTTTATTTTTTAATAAAAAAAATATAGTATTATATAGTA...TAA'), id='NC_004325.2-100000', name='NC_004325.2-100000', description='NC_004325.2-100000', dbxrefs=[]),
 SeqRecord(seq=Seq('TATATCTTTAAAATGATGTTGCAAATTTATTGAACATGTTAATAAATCATCCTG...TCA'), id='NC_004325.2-99999', name='NC_004325.2-99999', description='NC_004325.2-99999', dbxrefs=[]),
 SeqRecord(seq=Seq('TCATGATTTACATATATTTGTAAAACATATATAATCTGTCCAGACATATTATAT...CTC'), id='NC_004325.2-99998', name='NC_004325.2-99998', description='NC_004325.2-99998', dbxrefs=[]),
 SeqRecord(seq=Seq('AATTAATAATTATTATTATTACATATATATTTGTTATGTTTTGTTATATAATAT...TGT'), id='NC_004325.2-99997', name='NC_004325.2-99997', description='NC_004325.2-99997', dbxrefs=[]),
 SeqRecord(seq=Seq('ATGCTTTTTAAAAATGAAGACAGTGGCAATGGTGTTTACGGTTTTACTTATAAT...ATG'), id='NC_004325.2-99996', name='NC_004325.2-99996', description='NC_004325.2-99996', dbxrefs=[]),
 SeqRecord(seq=Seq('AAAATATTAATATAGAAATGTTAAAAATACGAATCAATTATATTTGTGATAAAG...TAT'), id='NC_004325

In [37]:
Genome

[SeqRecord(seq=Seq('TGAACCCTaaaacctaaaccctaaaccctaaaccctgaaccctaaaccctgaac...agg'), id='NC_004325.2', name='NC_004325.2', description='NC_004325.2 Plasmodium falciparum 3D7 genome assembly, chromosome: 1', dbxrefs=[]),
 SeqRecord(seq=Seq('aaccctaaaccctaaaccctaaaccctaaaccctaaaccctaaacctaaaccct...TCA'), id='NC_037280.1', name='NC_037280.1', description='NC_037280.1 Plasmodium falciparum 3D7 genome assembly, chromosome: 2', dbxrefs=[]),
 SeqRecord(seq=Seq('TAAACCCTAAATCTCTAAACCCTAAAGCTATACCTAAACCCTGAAGGTTATACC...TCA'), id='NC_000521.4', name='NC_000521.4', description='NC_000521.4 Plasmodium falciparum 3D7 genome assembly, chromosome: 3', dbxrefs=[]),
 SeqRecord(seq=Seq('aaccctaaaccctgaaccctaaaccctaaaccctgaaccctgaaccctaaaccc...tta'), id='NC_004318.2', name='NC_004318.2', description='NC_004318.2 Plasmodium falciparum 3D7 genome assembly, chromosome: 4', dbxrefs=[]),
 SeqRecord(seq=Seq('ctaaaccctgaaccctaaaccctgaaccctaaaccctaaaccctgaaccctaaa...ggt'), id='NC_004326.2', name='NC_004326.2', de

In [38]:
len(reads[0].seq)

100

In [39]:
testG = Genome[0].seq[0:int(1e4)]

In [40]:
BWTg, SATg = BWT_suffix_table(testG)

In [41]:
searching(BWTg, SATg, reads[0].seq)

(-1, 'Word not in text')

Découpons le premier chromosome en bouts de longueur `n` avec un overlap de `k`.

In [8]:
def cutSequence(seqToCut, n, k):
    cutSeq = []
    iF = 0
    iL = iF + n

    Continuer = True

    while Continuer:
        if iL >= len(seqToCut):
            Continuer = False
            iL = len(seqToCut)
        cutSeq.append(seqToCut[iF:iL])
        iF += n - k
        iL = iF + n -1
        if Continuer and len(seqToCut) - iF <= k :
            Continuer = False
            
    return cutSeq

In [9]:
def searchWord(cutSeq, n, k, words):

    bwtSeq = []
    satSeq = []
    for s in cutSeq:
        bwt, sat = BWT_suffix_table(s)
        bwtSeq.append(bwt)
        satSeq.append(sat)
        
    foundRead = []
    for i in range(len(cutSeq)):
        res = searching(bwtSeq[i], satSeq[i], words)
        if res[0] != -1 :
            resi = [x+i*(n-k) for x in res]
            foundRead += resi
            
    return np.unique(foundRead)

In [64]:
seqToCut = 'abaghstbababbagtdjaba'
n = 11
words = 'aba'
k = len(words)

repTest = [0,8,18]

In [65]:
cutSeq = cutSequence(seqToCut, n, k)

In [66]:
cutSeq

['abaghstbaba', 'ababbagtdj', 'djaba']

In [67]:
foundWord = searchWord(cutSeq, n, k, words)

In [68]:
foundWord

array([ 0,  8, 18])

In [69]:
for i in foundWord:
    print(seqToCut[i:i+k])

aba
aba
aba


In [44]:
sum(cut_search(seqToCut, n, words) == repTest) == len(repTest)

True

Les fonctions fonctionnent sur la séquence test. Essayons sur le chromosome 1 :

In [76]:
chr1 = Genome[0].seq.upper()
n = int(1e4)
read1 = reads[0].seq.upper()
k1 = len(read1)

In [89]:
k1

100

In [77]:
cutChr1 = cutSequence(chr1, n, k1)

In [78]:
cutChr1

[Seq('TGAACCCTAAAACCTAAACCCTAAACCCTAAACCCTGAACCCTAAACCCTGAAC...TAC'),
 Seq('TGACTACTAACATGATCAGTAACATGACTACTAACATCATCATAACTAACATGA...CTG'),
 Seq('GATCTTACTTTCACTAACTTAGGTCTTACTTTTGCTAACATAGGTCTTACTTTC...AGA'),
 Seq('CTATAAACCATGTGCCCTTGAATATGAATATTATAAGCATACTAATGGCGGTGG...TTA'),
 Seq('GGAATGGAGTCACTTTGTATATCAGTTTGTAATGTGGCAAATTTGTCCATTAAT...TAA'),
 Seq('ACACTACATAAAACTATACAATAGTATTTTATTAATCTTAATAATTTCTTTTTT...GAT'),
 Seq('TATTGTAAAATATAAAACAATATTTTAATGGTAATTTATCGTATGACATAAAAT...GCT'),
 Seq('TATTTATGTATCACTTTTATTTTCTGTTTTATCTTTACCCATCAATTTCCCTCT...AAA'),
 Seq('GTACCAGATAAACAAGGTTACTTACGACATAAATTTGAATGTTACCGAAAATAT...AGT'),
 Seq('ATTACATATATTTATTGTATTTATATAATAAAAAATATAATATATTATTTGTAC...TTA'),
 Seq('TATCATTTTAAATGTAAGTTTTTTTTTTTTTTTTTTTTTTTGAAATAAAATACA...TTT'),
 Seq('TAAATATTTATTTATTTTAAGTGGAAATATGCGGAAATTATTGAAAAAAAAAAA...TTG'),
 Seq('AAAATATAAATATAAATATAAAGAAAAATATAAAGAAAAATATAAAAAAACAAT...GTA'),
 Seq('TTATTTTCCTCTTAGTTTAATTTAAATAAAAATGGGTAAACAAGCATAACACAA...ATC'),
 Seq('ATTTTCTATCTTTT

In [79]:
foundRead1 = searchWord(cutChr1, n, k1, read1)

In [80]:
foundRead1

array([], dtype=float64)

On va faire une seule fonction pour rassembler les deux étapes (cut et search)

In [10]:
def cut_search(seqToCut, n, word):
    k = len(word)
    cutSeq = cutSequence(seqToCut, n, k)
    foundWord = searchWord(cutSeq, n, k, word)
    return foundWord

On va chercher le premier read dans chaque chromosome

In [86]:
n = int(1e4)
read1 = reads[0].seq.upper()
foundRead = {i:[] for i in range(len(Genome))}

for i in range(len(Genome)):
    chromosome = Genome[i].seq.upper()
    found = cut_search(chromosome, n, read1)
    print(found)
    foundRead[i] = found

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]


In [90]:
foundRead

{0: array([], dtype=float64),
 1: array([], dtype=float64),
 2: array([], dtype=float64),
 3: array([], dtype=float64),
 4: array([], dtype=float64),
 5: array([], dtype=float64),
 6: array([], dtype=float64),
 7: array([], dtype=float64),
 8: array([], dtype=float64),
 9: array([], dtype=float64),
 10: array([], dtype=float64),
 11: array([], dtype=float64),
 12: array([], dtype=float64),
 13: array([], dtype=float64),
 14: array([], dtype=float64)}

Y'a R

In [ ]:
n = int(1e4)
foundAll = {j:{} for j in range(len(reads))}

for j in range(len(reads)):
    print("read ",j)
    read = reads[j].seq.upper()
    foundRead = {i:[] for i in range(len(Genome))}

    for i in range(len(Genome)):
        print("chr",i+1)
        chromosome = Genome[i].seq.upper()
        found = cut_search(chromosome, n, read)
        print(found)
        foundRead[i] = found
        
    foundAll[j] = foundRead

read  0
chr 1
[]
chr 2
[]
chr 3
[]
chr 4
[]
chr 5
[]
chr 6
[]
chr 7
[]
chr 8
[]
chr 9
[]
chr 10
[]
chr 11
[]
chr 12
[]
chr 13
[]
chr 14
[]
chr 15
[]
read  1
chr 1
[]
chr 2
[]
chr 3
[]
chr 4
[]
chr 5
[]
chr 6
[]
chr 7
[]
chr 8
[]
chr 9
[]
chr 10
[]
chr 11
[]
chr 12
[]
chr 13
[]
chr 14
[]
chr 15
[]
read  2
chr 1
[471737]
chr 2
[]
chr 3
[]
chr 4
[]
chr 5
[]
chr 6
[]
chr 7
[]
chr 8
[]
chr 9
[]
chr 10
[]
chr 11
[]
chr 12
[]
chr 13
[]
chr 14
[]
chr 15
[]
read  3
chr 1
[]
chr 2
[]
chr 3
[]
chr 4
[]
chr 5
[]
chr 6
[]
chr 7
[]
chr 8
[]
chr 9
[]
chr 10
[]
chr 11
[]
chr 12
[]
chr 13
[]
chr 14
[]
chr 15
[]
read  4
chr 1
[]
chr 2
[]
chr 3
[]
chr 4
[]
chr 5
[]
chr 6
[]
chr 7
[]
chr 8
[]
chr 9
[]
chr 10
[]
chr 11
[]
chr 12
[]
chr 13
[]
chr 14
[]
chr 15
[]
read  5
chr 1
[]
chr 2
[]
chr 3
[]
chr 4
[]
chr 5
[]
chr 6
[]
chr 7
[]
chr 8
[]
chr 9
[]
chr 10
[]
chr 11
[]
chr 12
[]
chr 13
[]
chr 14
[]
chr 15
[]
read  6
chr 1
[592074]
chr 2
[]
chr 3
[]
chr 4
[]
chr 5
[]
chr 6
[]
chr 7
[1390936]
chr 8
[]
chr 9
[

## BAM

In [47]:
!pip install pysam

In [88]:
import pysam

samfile = pysam.AlignmentFile('./single_Pfal_dat.bam', 'rb')

iter = samfile.fetch(until_eof=True)

In [ ]:
for x in iter:
    print(str(x))